In [3]:
import numpy as np
import xarray as xr
import pandas as pd
import glob

In [4]:
filelist = sorted(glob.glob('/Users/ahutson/research/rand_era5_data/era5_*'))
print(filelist)

['/Users/ahutson/research/rand_era5_data/era5_2019.nc', '/Users/ahutson/research/rand_era5_data/era5_2020.nc', '/Users/ahutson/research/rand_era5_data/era5_2021.nc']


In [30]:
for f,file in enumerate(filelist):
    print(file)
    fh = xr.open_dataset(file)
    lat,lon = fh.latitude.data,fh.longitude.data
    time = fh.time.data
    
    t2m = fh.t2m.data
    tp  = fh.tp.data
    
    # find gridpoint closest to CIGLR coordinates
    ciglr = [42.2773, -83.7366]
    clat  = np.argwhere(np.abs(lat-ciglr[0])<0.1)[0]
    clon  = np.argwhere(np.abs(lon-ciglr[1])<0.1)[0]
    
    # get temperature,precip from ciglr, append to total array
    if f==0:
        time_all= time
        t2m_all = t2m[:,clat,clon][:,0]
        tp_all  = tp[:,clat,clon][:,0]
    else:
        time_all= np.concatenate((time_all,time))
        t2m_all = np.concatenate((t2m_all,t2m[:,clat,clon][:,0]))
        tp_all  = np.concatenate((tp_all,tp[:,clat,clon][:,0]))
    print(tp_all.shape)
    
    

/Users/ahutson/research/rand_era5_data/era5_2019.nc
(728,)
/Users/ahutson/research/rand_era5_data/era5_2020.nc
(1460,)
/Users/ahutson/research/rand_era5_data/era5_2021.nc
(2188,)


In [43]:
d  = {'time':time_all,'t2':t2m_all,'precip':tp_all}
df = pd.DataFrame(data=d)
df = df.set_index('time')
df.to_csv('era5_ciglr_temp.nc')